# Part 1

In [1]:
import pandas as pd

#From LCFS
households = pd.read_csv("lcfs_2019_dvhh_ukanon.tab", delimiter="\t")
persons = pd.read_csv("lcfs_2019_dvper_ukanon201920.tab", delimiter="\t")
spending = pd.read_csv("lcfs_2019_dv_set89_ukanon.tab", delimiter="\t") #spending is weekly, adult spenders only

#From NCFS (this is a csv of just the one row we need)
emissions = pd.read_csv("emissions.csv") #Total emissions for each NCFS category in Ktonnes

/Users/inesfernandez/opt/anaconda3/envs/openfisca/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (1872,1875) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Codes for larger categories
categories = {"Food and non-alcoholic beverages": "1.", 
"Alcoholic beverages, tobacco and narcotics": "2.",
"Clothing and footwear": "3.",
"Housing, water, electricity, gas and other fuels": "4.", 
"Furnishings, household equipment and routine maintenance of house": "5.",
"Health": "6.",
"Transport": "7.",
"Communication": "8.",
"Recreation and culture": "9.",
"Education": "10",
"Restaurants and hotels": "11",
"Miscellaneous goods and services": "12",
#"Non-consumption expenditure": "20"
}

#Change codes in spending df (originally for super specific categories) to codes for larger categories
spending["COI_PLUS"] = spending["COI_PLUS"].transform(lambda x: x[:2])

#If code is 20, change to 12 to count non-consumption expenditure as misc goods & services
spending["COI_PLUS"].replace({"20": "12"}, inplace=True)

In [3]:
#Total expenditure for larger LCFS categories
y = spending.groupby(["COI_PLUS"]).pdamount.sum()
total_expenditure = {}
for n in categories.keys():
    value = y.loc[categories[n]] * 52 #because weekly
    total_expenditure[n] = value

In [5]:
#NCFS categories included in each LCFS larger category
categories_emissions = {'Food and non-alcoholic beverages': ['Food', 'Non-alcoholic beverages'],
 'Alcoholic beverages, tobacco and narcotics': ['Alcoholic beverages', 'Tobacco'],
 'Clothing and footwear': ['Clothing', 'Footwear'],
 'Housing, water, electricity, gas and other fuels': [
     #'Actual rentals for households',
       'Imputed rentals for households', 
     'Electricity, gas and other fuels'
 'Water supply and miscellaneous dwelling services'],             
 'Furnishings, household equipment and routine maintenance of house': ['Furniture, furnishings, carpets etc', 'Household textiles',
       'Household appliances', 'Glassware, tableware and household utensils',
       'Tools and equipment for house and garden',
       'Goods and services for household maintenance'
        'Maintenance and repair of the dwelling'],
 'Health': ['Medical products, appliances and equipment', 'Hospital services'],
 'Transport': ['Purchase of vehicles', 'Operation of personal transport equipment',
       'Transport services'],
 'Communication': ['Postal services',
       'Telephone and telefax equipment', 'Telephone and telefax services'],
 'Recreation and culture': ['Audio-visual, photo and info processing equipment',
       'Other major durables for recreation and culture',
       'Other recreational equipment etc',
       'Recreational and cultural services','Newspapers, books and stationery'],
 'Education': ['Education'],
 'Restaurants and hotels': ['Restaurants and hotels'],
 'Miscellaneous goods and services': ['Miscellaneous goods and services']}

In [16]:
x = pd.melt(emissions)
x["LCFS cat"] = pd.Series(0)

#Assign LCFS categories to NCFS categories
for n in range(34):
    for cat in categories_emissions:
        if x["variable"].loc[n] in categories_emissions[cat]:
            x["LCFS cat"].loc[n] = cat

#Not sure why these don't appear automatically
x["LCFS cat"].loc[8] = 'Furnishings, household equipment and routine maintenance of house'
x["LCFS cat"].loc[9] = 'Housing, water, electricity, gas and other fuels'
x["LCFS cat"].loc[10] = 'Housing, water, electricity, gas and other fuels'
x["LCFS cat"].loc[16] ='Furnishings, household equipment and routine maintenance of house'
x = x.rename(columns={'value': 'emissions'})

#Group by LCFS category, find total emissions, expenditure, & emissions per pound (in Ktonnes)
x2 = x.groupby("LCFS cat").sum()
x2["expenditure"] = pd.Series(0)
for cat in total_expenditure:
    x2["expenditure"][x2.index == cat] = total_expenditure[cat]
    
x2["emissions per pound"] = x2["emissions"]/x2["expenditure"]
x2

,emissions,expenditure,emissions per pound
LCFS cat,,,
"Alcoholic beverages, tobacco and narcotics",877.641693,3.721298e+06,0.000236
Clothing and footwear,8060.591879,6.770790e+06,0.001190
Communication,2632.276247,6.556418e+05,0.004015
Education,1905.713172,1.160370e+05,0.016423
Food and non-alcoholic beverages,21736.370678,1.776790e+07,0.001223
"Furnishings, household equipment and routine maintenance of house",11887.009387,5.279863e+06,0.002251
Health,7271.687156,2.195981e+06,0.003311
"Housing, water, electricity, gas and other fuels",4361.813490,8.385346e+05,0.005202
Miscellaneous goods and services,15944.076170,1.570487e+07,0.001015


In [8]:
#Check that this and below match
x2["emissions"].sum()

416918.4047964

In [9]:
emissions["Total"] - emissions ["Actual rentals for households"]

0    416918.404753
dtype: float64

In [10]:
#Emissions for a household
def get_category_total(household, category):
    cat_code = categories.get(category)
    household_rows = spending.loc[spending["case"] == household]
    household_rows["COI_PLUS"] = household_rows["COI_PLUS"].transform(lambda x: x[:2])
    
    #if code is 20 change to 12; include non-consumption in misc goods & services
    household_rows["COI_PLUS"].replace({"20": "12"}, inplace=True)
    
    relevant_rows = household_rows.loc[household_rows["COI_PLUS"] == cat_code]
    
    #multiply by 52 to get annual
    expenditure = relevant_rows["pdamount"].sum() * 52
    return expenditure * x2["emissions per pound"][category]

def get_household_total(household):
    emissions = 0
    for n in categories:
        category_emissions = (get_category_total(household, n))
        emissions = emissions + category_emissions
    return emissions

In [13]:
#Example
pd.options.mode.chained_assignment = None
get_household_total(780)

64.80989757442732

In [14]:
#Add emissions column to households dataframe
households["Emissions"] = pd.Series(get_household_total(n) for n in households["case"])

In [15]:
#Check that this matches with above
households["Emissions"].sum()

416918.40479640313

# Part 2

In [ ]:
#LCFS variables using field names
def get_LCFS_variables():
    from openfisca_uk.entities import Person, BenUnit, Household
    
    #Household size
    class A049(Variable):
        value_type = float
        entity = Household
        definition_period = ETERNITY
    
    #Gross household income (weekly)
    class P352p(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
    
    #Equivalized income (McClements Scale)
    class EqIncDMp(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
        
    #Equivalised income (OECD Scale)
    class EqIncDOp(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
        
    #Location (gov office region)
    class Gorx(Variable):
        value_type = float
        entity = Household
        definition_period = ETERNITY 
        
    #Rent
    class B010(Variable):
        value_type = float
        entity = Household
        definition_period = ETERNITY

In [ ]:
def get_input_variables():
    
    #Household size
    class household_size(Variable):
        value_type = float
        entity = Household
        label = "Number of people in household"
        definition_period = ETERNITY

        def formula(household, period, parameters):
            return household("A049", period) * WEEKS_IN_YEAR
    
    #Gross household income
    class gross_income(Variable):
        value_type = float
        entity = Household
        label = "Gross household income"
        definition_period = YEAR

        def formula(household, period, parameters):
            return household("P352p", period) * WEEKS_IN_YEAR
    
    #Equivalized income (McClements Scale)
    class equivalized_income(Variable):
        value_type = float
        entity = Household
        definition_period = YEAR 
        
        def formula(household, period, parameters):
            return household("EqIncDMp", period) * WEEKS_IN_YEAR 
            #if OECD scale, replace with "EqIncDOp"
        
    class region(Variable):
        value_type = Enum
        possible_values = Region
        default_value = Region.UNKNOWN
        entity = Household
        label = "Region of the UK"
        definition_period = ETERNITY

        def formula(household, period, parameters):
            region = household("Gorx", period)
            reg = select(
                [
                    region == 1,
                    region == 2,
                    region == 4,
                    region == 5,
                    region == 6,
                    region == 7,
                    region == 8,
                    region == 9,
                    region == 10,
                    region == 11,
                    region == 12,
                    region == 13,
                ],
                [
                    Region.NORTH_EAST,
                    Region.NORTH_WEST,
                    Region.YORKSHIRE,
                    Region.EAST_MIDLANDS,
                    Region.WEST_MIDLANDS,
                    Region.EAST_OF_ENGLAND,
                    Region.LONDON,
                    Region.SOUTH_EAST,
                    Region.SOUTH_WEST,
                    Region.WALES,
                    Region.SCOTLAND,
                    Region.NORTHERN_IRELAND,
                ],
            )
            return reg
    
    class rent(Variable):
        value_type = float
        entity = Household
        label = "Gross rent for the household"
        definition_period = YEAR

        def formula(household, period, parameters):
            return household("B010", period) * WEEKS_IN_YEAR

In [ ]:
from openfisca_uk_data.datasets.frs.raw_frs import RawFRS
from pathlib import Path
from typing import List
from openfisca_core.model_api import *
from openfisca_uk_data.utils import dataset
import pandas as pd
import shutil
from openfisca_uk_data.utils import (
    CAPITAL_INCOME_VARIABLES,
    LABOUR_INCOME_VARIABLES,
    uprated,
)
import h5py
from openfisca_uk_data.datasets.frs.base_frs.dataset import BaseFRS
from openfisca_uk_data.datasets.frs.base_frs.model_input_variables import (
    get_input_variables,
)


def from_FRS(year: int = 2018):
    from openfisca_uk import CountryTaxBenefitSystem

    system = CountryTaxBenefitSystem()
    variables = []
    for variable in get_input_variables():
        try:
            variables += [type(system.variables[variable.__name__])]
        except:
            variables += [variable]
    for i in range(len(variables)):
        variable = variables[i]
        if variable.__name__ in LABOUR_INCOME_VARIABLES:
            variables[i] = uprated(
                "uprating.labour_income", from_year=year + 1
            )(variable)
        elif variable.__name__ in CAPITAL_INCOME_VARIABLES:
            variables[i] = uprated(
                "uprating.labour_income", from_year=year + 1
            )(variable)
        else:
            variables[i] = uprated(from_year=year + 1)(variable)

    class reform(Reform):
        def apply(self):
            for var in variables:
                self.update_variable(var)

    return reform


@dataset
class FRS:
    name = "frs"
    openfisca_uk_compatible = True
    input_reform_from_year = from_FRS

    def generate(year) -> None:
        base_frs_years = BaseFRS().years
        if len(base_frs_years) == 0:
            raw_frs_years = RawFRS().years
            if len(raw_frs_years) == 0:
                raise Exception("No FRS microdata to generate from")
            else:
                base_frs_year = max(raw_frs_years)
        else:
            base_frs_year = max(base_frs_years)
        from openfisca_uk import Microsimulation

        base_frs_sim = Microsimulation(dataset=BaseFRS, year=base_frs_year)
        person_vars, benunit_vars, household_vars = [
            [
                var.__name__
                for var in get_input_variables()
                if var.entity.key == entity
            ]
            for entity in ("person", "benunit", "household")
        ]
        with h5py.File(FRS.file(year), mode="w") as f:
            for variable in person_vars + benunit_vars + household_vars:
                f[f"{variable}/{year}"] = base_frs_sim.calc(
                    variable, year
                ).values

In [ ]:
from openfisca_uk_data.utils import (
    MAIN_INPUT_VARIABLES,
    dataset,
    uprate_variables,
)
import synthimpute as si
import numpy as np
import h5py

class FRS_LCFS_Adjusted:
    name = "frs_lcfs_adj"
    openfisca_uk_compatible = True
    input_reform_from_year = uprate_variables(MAIN_INPUT_VARIABLES)

    def generate(year):
        from openfisca_uk import Microsimulation
        from openfisca_uk_data.datasets.frs.frs import FRS
        LCFS = households
        

        frs_sim = Microsimulation(dataset=FRS)
        lcfs_sim = Microsimulation(dataset=LCFS)
        
        #common variables
        lcfs_common_variables = np.array(
            [
                lcfs_sim.calc("household_size", year).values,
                lcfs_sim.calc("gross_household_income", year).values,
                lcfs_sim.calc("equivalized_income", year).values,
                lcfs_sim.calc("region", year).values,
                lcfs_sim.calc("rent", year).values,
                
            ]
        ).T
        
        frs_common_variables = np.array(
            [
                frs_sim.calc("household_size", year).values,
                frs_sim.calc("gross_household_income", year).values,
                frs_sim.calc("equivalized_income", year).values,
                frs_sim.calc("region", year).values,
                frs_sim.calc("rent", year).values,
            ]
        ).T
        
        #imputed variables
        lcfs_emissions = lcfs_sim.calc("dividend_income", year).values
        frs_weight = frs_sim.calc("household_weight", year).values
        
        print(
            "Imputing emissions for FRS respondents from LCFS values...",
            end="",
        )
        imputed_emissions = si.rf_impute(
            x_train=lcfs_common_variables,
            y_train=lcfs_emissions,
            x_new=frs_common_variables,
            sample_weight_train=frs_weight,
            mean_quantile=0.18,
        )
        print(" completed.")
        imputed_emissions *= (
            frs_sim.calc("emissions", year).values > 0
        )
        frs_sim.simulation.set_input(
            "emissions", year, imputed_emissions
        )
        with h5py.File(
            FRS_SPI_Adjusted.data_dir / FRS_SPI_Adjusted.filename(year), "w"
        ) as f:
            for variable in MAIN_INPUT_VARIABLES:
                f[f"{variable}/{year}"] = frs_sim.calc(variable, year).values


In [ ]:
households["Emissions"].sum()